# Stopwords

Dieses Notebook nutzt die Programmiersprache *Python*, um in ihren Texten Stopwords zu entfernen oder nur diese zu behalten. Um ihnen etwas die Angst zu nehmen, führen wir jetzt vor allen anderen erst einmal ihre Feuertaufe durch.

Dazu klicken sie auf den Code in der nächsten Zelle. Dann drücken sie auf Windows STRG+ENTER, auf Apple CMD+ENTER.

In [13]:
print("Hello World!")

Hello World!


In [14]:
import os

# Aktuelles Arbeitsverzeichnis anzeigen
print("Aktuelles Verzeichnis:", os.getcwd())

Aktuelles Verzeichnis: C:\Users\jwieb\Documents\Normalisierungsordner


In [15]:
# Neues Arbeitsverzeichnis setzen
os.chdir(r'C:\Users\jwieb\Documents\Normalisierungsordner')

In [16]:
print("Neues Verzeichnis:", os.getcwd())

Neues Verzeichnis: C:\Users\jwieb\Documents\Normalisierungsordner


In [17]:
!pip install more-itertools


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Herzlichen Glückwunsch, sie haben nun Code ausgeführt, nämlich das berühmte "Hello World!"-Programm. Sie sind jetzt alle Programmierer.


## Ein paar Bemerkungen zu Colab
Colab ist generell kostenlos, hat allerdings einige Eigenheiten, die es zu beachten gilt.
1. Es ist von Google
2. Sie müssen immer alle Daten hochladen, Colab merkt sich die letzte Sitzung nicht
3. Das heißt auch, dass sie alles wieder herunterladen müssen (wird hier automatisch gemacht)  
4. Wenn sie zulange inaktiv sind, wird die Sitzung beendet und sie müssen von vorne anfangen

## Erste Schritte

1. Klicken sie auf das Ordnersymbol auf der linken Seite.
2. Rechtsklicken sie in der sich öffnenden Leiste und wählen sie "New Folder".
3. Benennen sie den Ordner *corpus-raw* (Kleinschreibung und "-" beachten, ansonsten funktioniert der Code nicht!).
4. Rechtsklicken sie ihren neuen Ordner, wählen sie "Upload".
5. In dem sich öffnenden Fenster suchen sie ihre *.txt*-Datein, wählen sie alle aus und klicken sie auf "Öffnen".
6. Wiederholen sie Schritt 2 und 3, benennen sie den neuen Ordner *corpus-clean*

Nun sollten alle ihre Daten hochgeladen sein. Falls nicht, einfach rufen!

Klicken sie nun auf den Code in der nächsten Zelle und führen sie diesen aus.


In [18]:
import time
from typing import Union
import warnings
import requests
from more_itertools import divide

CAB_URL = 'https://www.deutschestextarchiv.de/public/cab/query?clean=1&qname=q&a=default&fmt=raw&file=C%3A%5Cfakepath%5Ctest.txt'
CAB_HEADERS = headers = {'Content-Type': 'text/plain'}

def cab(text: str, delay: Union[float, None] = None) -> Union[str, None]:

    """
    Queries the CAB-Webservice provided by Deutsches Textarchiv for orthographic normalisation.
    If you use this function repeatedly (e.g. in a loop), please use with delay parameter to avoid overloading the server.
    If a texts exceeds the size of one megabyte, it is split into smaller parts
    and then sent to the service iteratively.
    """

    if delay is not None:
        time.sleep(delay)

    n_megabytes = len(text.encode('utf-8')) // 1000000

    if n_megabytes >= 1:
        parts = list(divide(n=n_megabytes+1, iterable=text.split(' ')))
        parts = [' '.join(part) for part in parts]
    else:
        parts = [text]

    normed_parts = []
    for part in parts:
        r = requests.post(url=CAB_URL, headers=CAB_HEADERS,
                          data=part.encode('UTF-8'))

        if r.status_code != 200:
            warnings.warn(
                f'Request returned with error code {r.status_code}\nError Message\n{r.body}')
            return None

        normed_parts.append(r.text.strip())

    return ' '.join(normed_parts)

Sie haben nun einen eigenen Algorithmus erstellt, den sie per *cab()* verwenden können.

Führen sie nun die nächste Zelle aus.

In [19]:
import os

corpus_raw = []
filenames = []

for file in os.listdir("corpus-raw"):
  try:
    with open("corpus-raw/" + file, "r", encoding="utf8") as f:
      corpus_raw.append(f.read())
      filenames.append(file)
  except Exception as e:
    print("Error with file:\t" + file)
    print(e)

Der Code ist relativ simpel: Alle *.txt*-Datein im Ordner *corpus-raw* werden in eine Liste namens*corpus_raw* eingelesen. Gleichzeitig werden die Dateinamen, die sie gewählt haben, in eine weitere Liste *filenames* eingelesen. In der nächsten Zelle werden der erste Dateiname und die ersten 100 Zeichen der ersten Datei ausgegeben.

**Aufgabe:** Was müssen sie her verändern, wenn sie den Dritten Dateinamen und die ersten 100 Zeichen ihres dritten Textes ausgeben wollen?

In [20]:
print(filenames[0])
print(corpus_raw[0][:100])

arnima_invalide_1818.txt
Gaben der Milde.
Viertes Bändchen.
Mit Beiträgen von L. Achim von Arnim, F. W. Gubitz, Heraklius, C.


Wenn sie mit der oberen Zelle etwas herumgespielt haben, kommen wir nun zu dem interessante Part: dem Aufbereiten der Texte. Unter Normalisierung wird der Prozess verstanden, die Rechtschreibung ihrer Texte anzupassen.



In [21]:
corpus_clean = []

for text in corpus_raw:
  corpus_clean.append(cab(text, delay=3.0))

Danach lassen wir uns erneut den ersten Dateinamen anzeigen und dieses Mal den bereinigten Text der ersten Datei. So können wir überprüfen, ob unser Code funktioniert.

In [22]:
print(filenames[0])
print(corpus_clean[0][:1000])

arnima_invalide_1818.txt
Gaben der Milde .
Viertes Bändchen .
Mit Beiträgen von L. Achim von Arnim , F. W. Gubitz , Heraklius , C. F. E. Ludwig .
Wilh. Müller , L. Velhar und Julius von Voß .
Für die Bücher-Verloosung " zum Vorteil hilfloser Krieger " herausgegeben von F. W. Gubitz .
Berlin , 1818 .
Der tolle Invalide auf dem Fort Ratonneau .
Erzählung von L. Achim von Arnim .
Graf Dürande , der gute alte Kommandant von Marseille , saß einsam frierend an einem kalt stürmenden Oktoberabende bei dem schlecht eingerichteten Kamine seiner prachtvollen Kommandantenwohnung und rückte immer näher und näher zum Feuer , während die Kutschen zu einem großen Balle in der Straße vorüber rollten , und sein Kammerdiener Basset , der zugleich sein liebster Gesellschafter war , im Vorzimmer heftig schnarchte .
Auch im südlichen Frankreich ist es nicht immer warm , dachte der alte Herr , und schüttelte mit dem Kopfe , die Menschen bleiben auch da nicht immer jung , aber die lebhafte gesellige Bewegung 

Wenn sie zufrieden sind führen sie die nächste Codezelle aus.

Es werden hier einfach nur ihre aufbereiteten Texte in den Ordner *corpus-clean* mit den selben Namen wie ihre gesammelten Texte gespeichert.

In [23]:
for i in range(len(filenames)):
  try:
    with open("corpus-clean/" + filenames[i], "w", encoding="utf8") as f:
      f.write(corpus_clean[i])
  except Exception as e:
    print("Error with file:\t" + file)
    print(e)

Frustrierenderweise kann man nicht bequem über Colab einen gesamten Ordner herunterladen (und sie werden sicherlich nicht ihre 50+ Texte manuell herunterladen!). Deswegen packen wir den Ordner *corpus-clean* in ein Zip-Archiv. Führen sie hierzu einfach die nächste Zelle aus (sie können den Output ignorieren).

In [28]:
!zip -r corpus.zip /content/corpus-clean

Der Befehl "zip" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


Mit der nächsten Zelle wird das Zip-Archiv heruntergeladen:

In [29]:
from google.colab import files

files.download("corpus.zip")

ModuleNotFoundError: No module named 'google'

Sie haben nun ihre Texte aufbereitet und dafür vielleicht (wenn die Technik so mitgemacht hat) fünf Minuten gebraucht.